Plotting example.

The plotting package allows you to make an interactive CRN plot. The CRN plot has a default representation:
* species are circles
    - orange circles are RNA species
    - blue circles are complexes
    - green circles are proteins
    - grey circles are DNA
    -there's always a purple circle that represents nothing. If your CRN doesn't have reactions that go to nothing, then it won't have any connections.
    
* reactions are squares

Click on a node (either reaction or species) and all arrows including that node will be highlighted
Mouse over a node (either reaction or species) and a tooltip will tell you what it is.

In [1]:
%matplotlib inline

from biocrnpyler import *
import numpy as np

txtl = CRNLab("GFP")
txtl.mixture("mixture1", extract = "TxTlExtract", mixture_volume = 1e-6, mixture_parameters = 'BasicExtract.tsv')
dna = DNAassembly("mydna",promoter=RegulatedPromoter("plac",["laci"]),rbs="UTR1",protein="GFP")
txtl.add_dna(name = "G1", promoter = "pBest", rbs = "BCD2", protein = "GFP", initial_conc = 10, volume = 1e-7)
well1 = txtl.get_model()

In [2]:
from bokeh.models import (Plot , Range1d)
import bokeh.plotting
import bokeh.io
bokeh.io.output_notebook()
DG, DGspec, DGrxn = generate_networkx_graph(well1)
plot = Plot(plot_width=500, plot_height=500, x_range=Range1d(-500, 500), y_range=Range1d(-500, 500))
graphPlot(DG,DGspec,DGrxn,plot)
bokeh.io.show(plot)

C:\Users\andrey\Anaconda3\lib\site-packages\bokeh\resources.py:107: ResourceWarning: unclosed file <_io.TextIOWrapper name='C:\\Users\\andrey\\Anaconda3\\lib\\site-packages\\bokeh\\_sri.json' mode='r' encoding='cp1252'>
  _SRI_HASHES = json.load(open(join(ROOT_DIR, "_sri.json")))


Loading BokehJS ...

C:\Users\andrey\Anaconda3\lib\site-packages\bokeh\plotting\graph.py:120: UserWarning: Node keys in 'layout_function' don't match node keys in the graph. These nodes may not be displayed correctly.
  warn("Node keys in 'layout_function' don't match node keys in the graph. "
